In [1]:
import sys

sys.path.append("../src")

import os

from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

from application.llm_service import LLMService
from infrastructure.text_embedding_pipeline import VectorStore
from application.ragas_evaluator import RAGASEvaluator

api_key = os.getenv("OPENAI_API_KEY")
file_path = "../data/example.pdf"

/Users/samuelkoh/projects/ragas_evaluation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
faiss_vector_store = VectorStore(api_key)
open_ai_llm = LLMService(api_key)

/Users/samuelkoh/projects/ragas_evaluation/notebooks/../src/infrastructure/text_embedding_pipeline.py:32: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  self.embedding_model = OpenAIEmbeddings(api_key=api_key)


In [3]:
query_answer_pairs = {
    "What is the product?": "GROW",
}
ragas_evaluator = RAGASEvaluator(
    vector_store=faiss_vector_store,
    llm_service=open_ai_llm,
    query_answer_pairs=query_answer_pairs,
)

In [4]:
ragas_evaluator.create_evaluation_dataset(top_k=3)

Retrieved 2 relevant documents for query: What is the product?
Error generating response: 'AIMessage' object has no attribute 'choices'


In [5]:
ragas_evaluator.evaluate(
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()]
)

Evaluating: 100%|██████████| 3/3 [00:05<00:00,  1.74s/it]


{'context_recall': 0.0000, 'faithfulness': 0.0000, 'factual_correctness(mode=f1)': 0.0000}

In [10]:
ragas_evaluator.evaluation_dataset.samples[0]

SingleTurnSample(user_input='What is the product?', retrieved_contexts=['2        None  None  Start a live chat at ourwebsite https://singli...  None\n3        None  None                               service@singlife.com  None\n4        None  None                              hotline: +65 69111111  None', '0     1                                                  2     3\n0  N eedhelp?  If y  ouneedhelp andanswersto yourpolicy,getin touch...      \n1        None  None                           https://faq.singlife.com  None'], reference_contexts=None, response='An error occurred while generating the response.', multi_responses=None, reference='GROW', rubrics=None)

In [ ]:
answers = faiss_vector_store.retrieve_relevant_text(
    "I need to withdraw money from my account, how can I do that?", top_k=5
)

In [19]:
help(faiss_vector_store.vector_store)

Help on FaissVectorStore in module infrastructure.text_embedding_pipeline object:

class FaissVectorStore(builtins.object)
 |  FaissVectorStore(embedding_model: infrastructure.text_embedding_pipeline.EmbeddingModel, index_path: str = '../ragas_evaluation/faiss_index') -> None
 |
 |  Handles FAISS-based vector storage and retrieval.
 |
 |  Methods defined here:
 |
 |  __init__(self, embedding_model: infrastructure.text_embedding_pipeline.EmbeddingModel, index_path: str = '../ragas_evaluation/faiss_index') -> None
 |      Initialize FAISS store with an embedding model and index path.
 |
 |  load_index(self) -> None
 |      Loads the FAISS index from disk.
 |
 |  query_text(self, query: str, top_k: int) -> List[str]
 |      Retrieves the most relevant chunks from FAISS.
 |
 |  store_text_chunks(self, chunks: List[str]) -> None
 |      Stores the text chunks in FAISS after generating embeddings.
 |
 |  ----------------------------------------------------------------------
 |  Data descript

In [ ]:
from infrastructure.pdf_parser import PDFParser
from infrastructure.text_embedding_pipeline import TextChunker

In [24]:
pdf_doc = PDFParser(file_path)
parsed_data = pdf_doc.parse()
text = parsed_data.get("text", "")
tables = parsed_data.get("tables", [])

In [26]:
text_chunker = TextChunker()

16939